In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import time
import os
import pandas as pd
from datetime import datetime



In [2]:
driver_path = "geckodriver.exe"  # Atualize o caminho
download_dir = r"C:\Users\andre\OneDrive - Mogai Tecnologia da Informação SA\Área de Trabalho\teste\IBOVDia"

# Configuração do Selenium
firefox_options = Options()
firefox_options.add_argument("--headless")  # Executa o navegador em modo headless

# Configurar diretório de download do Firefox
firefox_options.set_preference("browser.download.folderList", 2)  # 2 significa usar um diretório personalizado
firefox_options.set_preference("browser.download.dir", download_dir)
firefox_options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/csv")  # Defina o tipo MIME esperado para o CSV

# Iniciar o driver
service = Service(driver_path)
driver = webdriver.Firefox(service=service, options=firefox_options)

# URL da página
url = "https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br"



In [ ]:

try:
    driver.get(url)
    time.sleep(5)  # Espera carregar o conteúdo

    try:
        download = driver.find_element(By.XPATH, '//a[@href="/indexPage/day/IBOV?language=pt-br"]')
        if download.is_enabled():
            download.click()
        else:
            print("Link não está habilitado.")
    except Exception as e:
        print("Botão de download não encontrado ou desativado.")
        exit()

    time.sleep(10)  # Ajuste conforme necessário para garantir que o CSV seja baixado

    csv_files = [f for f in os.listdir(download_dir) if f.startswith('IBOVDia_') and f.endswith('.csv')]

    if csv_files:
        latest_file = max(csv_files, key=lambda f: datetime.strptime(f, 'IBOVDia_%d-%m-%y.csv'))
        data = latest_file.split("_")[1].split(".")[0]
        latest_file_path = os.path.join(download_dir, latest_file)

        df = pd.read_csv(latest_file_path, encoding='ISO-8859-1', sep=';', header=1, index_col=None)
        df.reset_index(inplace=True)
        columns = df.columns
        columns = df.columns[1:].to_list()
        columns.append("nan")
        df.columns = columns
        if df.iloc[:, -1].isna().all():
            df = df.iloc[:, :-1]
            
        df['date'] =  data
        df['date'] = pd.to_datetime(df['date'])

        parquet_file = os.path.join(download_dir, 'ibov_data.parquet')
        df.to_parquet(parquet_file, partition_cols=['date'], engine='pyarrow')
        os.remove(latest_file_path)

    else:
        print("Nenhum arquivo CSV encontrado!")

except Exception as e:
    print(f"Erro ao obter dados: {e}")

finally:
    driver.quit()

Cliquei no link para baixar o CSV.
Arquivo CSV mais recente: C:\Users\andre\OneDrive - Mogai Tecnologia da Informação SA\Área de Trabalho\teste\IBOVDia\IBOVDia_11-12-24.csv
CSV lido com sucesso!
                               Código        Ação                 Tipo  \
ALOS3                           ALLOS  ON      NM          502.481.592   
ALPA4                      ALPARGATAS  PN      N1          166.460.180   
ABEV3                       AMBEV S/A          ON        4.394.835.131   
ASAI3                           ASSAI  ON      NM        1.349.687.675   
AURE3                           AUREN  ON      NM          291.727.616   
...                               ...         ...                  ...   
VIVA3                     VIVARA S.A.  ON      NM          125.823.025   
WEGE3                             WEG  ON      NM        1.269.032.828   
YDUQ3                      YDUQS PART  ON      NM          290.010.465   
Quantidade Teórica Total          NaN         NaN       96.387.67

In [23]:
df = pd.read_csv(latest_file_path, encoding='ISO-8859-1', sep=';', header=1, index_col=0)
df.reset_index(inplace=True)
columns = df.columns
columns = df.columns[1:].to_list()
columns.append("nan")
df.columns = columns
if df.iloc[:, -1].isna().all():
    df = df.iloc[:, :-1]
df

,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
0,ALOS3,ALLOS,ON NM,502.481.592,"0,467"
1,ALPA4,ALPARGATAS,PN N1,166.460.180,"0,049"
2,ABEV3,AMBEV S/A,ON,4.394.835.131,"2,979"
3,ASAI3,ASSAI,ON NM,1.349.687.675,"0,406"
4,AURE3,AUREN,ON NM,291.727.616,"0,133"
...,...,...,...,...,...
83,VIVA3,VIVARA S.A.,ON NM,125.823.025,"0,136"
84,WEGE3,WEG,ON NM,1.269.032.828,"3,428"
85,YDUQ3,YDUQS PART,ON NM,290.010.465,"0,122"
86,Quantidade Teórica Total,NaN,NaN,96.387.676.727,"100,000"
